<a href="https://colab.research.google.com/github/yuliaplaksina/Machine-learning/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightgbm

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor


In [3]:
import io
from google.colab import files
#store = files.upload()

Saving googleplaystore.csv to googleplaystore (1).csv


In [4]:
store = pd.read_csv('googleplaystore.csv', sep=',')
store.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [5]:
store.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

### Делаем категориальными колонки, которые содержат не числа

In [0]:

for column in store.select_dtypes(include = ['object']).columns.tolist():
    store[column] = store[column].astype('category')

In [7]:
lgbm_regressor = lightgbm.LGBMRegressor().fit(store.loc[:, store.columns != 'Rating'], store['Rating'])
lgbm_regressor

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [8]:
list_of_importances = list(zip(store.loc[:, store.columns != 'Rating'].columns.tolist(), 
                               lgbm_regressor.feature_importances_))
sorted(list_of_importances, key= lambda x: x[1], reverse= True)

[('Installs', 244),
 ('App', 0),
 ('Category', 0),
 ('Reviews', 0),
 ('Size', 0),
 ('Type', 0),
 ('Price', 0),
 ('Content Rating', 0),
 ('Genres', 0),
 ('Last Updated', 0),
 ('Current Ver', 0),
 ('Android Ver', 0)]

In [9]:
important_features = [x[0] for x in sorted(list_of_importances, key= lambda x: x[1], reverse= True)[:1]]
important_features

['Installs']

In [0]:
important_features.extend(['Rating'])
store = store[important_features]

### Удаляем пропуски

In [0]:

for column in store.select_dtypes(include = ['int64', 'float64']).columns.tolist():
    store[column] = store[column].fillna(store[column].mean())
for column in store.select_dtypes(include = ['category']).columns.tolist():
    store[column] = store[column].fillna(store[column].describe(include= ['category'])['top'])

### Закодируем категориальные признаки

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in store.select_dtypes(include = ['category']).columns.tolist():
    le.fit(store[column])
    store[column] = le.transform(store[column])

store

,Installs,Rating
0,7,4.100000
1,19,3.900000
2,14,4.700000
3,17,4.500000
4,10,4.300000
5,16,4.400000
6,16,3.800000
7,4,4.100000
8,4,4.400000
9,7,4.700000


### Нормирование и масштабирование данных

In [13]:
store.loc[:, store.columns != 'Rating'] = store.loc[:, store.columns != 'Rating'].apply(lambda x: x/x.max(), axis=0)

store

,Installs,Rating
0,0.333333,4.100000
1,0.904762,3.900000
2,0.666667,4.700000
3,0.809524,4.500000
4,0.476190,4.300000
5,0.761905,4.400000
6,0.761905,3.800000
7,0.190476,4.100000
8,0.190476,4.400000
9,0.333333,4.700000
